In [1]:
import tensorflow as tf
import os
import collections
import numpy as np

# TODO(zongheng): visualization via tensorboard.
# TODO(zongheng): print some prediction from test set

# With gaussian weights.
# 500 training pts; 1000 testing.
PATH = "5rel-10numAttrs-22totalAttrs-100maxTblSize-10buckets-3000.csv"
NUM_FIELDS = 34
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [64,32]; lr = 1e-2
net = [64, 16, 64]; lr = 5e-3
net = [64,64]; lr =  1e-3
batch_size = 500
num_steps = 5500

# With random op types.
PATH = "5rel-10numAttrs-28totalAttrs-100maxTblSize-10buckets-3000-1526330982.csv"
NUM_FIELDS = 40
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [128,128]; lr =  5e-3
batch_size = 512

/Users/zongheng/anaconda/envs/ray-0321/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
def DatasetFromPath(pathname):
    dataset = tf.data.TextLineDataset(pathname)

    def ParseLine(line):
        fields = tf.decode_csv(line, [[0.0]] * NUM_FIELDS)
        # Last number is label.
        return tf.reshape(tf.stack(fields[:-1]), [-1]), fields[-1]

    def MeanCenterBatch(feat, lab):
        feat_mean, feat_var = tf.nn.moments(feat, axes=[0])
        lab_mean, lab_var = tf.nn.moments(lab, axes=[0])
        feat = (feat - feat_mean) / (tf.sqrt(feat_var) + 1e-8)
        lab = (lab - lab_mean) / (tf.sqrt(lab_var) + 1e-8)
        return feat, lab
    
#     # K-point dataset, for sanity checking.
#     K = 10
#     dataset = dataset.take(K).map(ParseLine)
#     dataset = dataset.shuffle(20000, reshuffle_each_iteration=False) \
#         .batch(K).map(MeanCenterBatch) \
#         .repeat()

    dataset = dataset.cache().map(ParseLine)
    dataset = dataset.shuffle(20000, reshuffle_each_iteration=True) \
        .repeat() \
        .batch(batch_size) #\
#         .map(MeanCenterBatch) 

    print(dataset.output_types)
    print(dataset.output_shapes)
    return dataset

# Train.
dataset = DatasetFromPath(TRAIN_PATH)
iterator = tf.data.Iterator.from_structure(
    dataset.output_types, dataset.output_shapes)
iterator_init = iterator.make_initializer(dataset)
# Test.
test_dataset = DatasetFromPath(TEST_PATH)
test_iterator_init = iterator.make_initializer(test_dataset)
# Shared.
feature_batch, label_batch = iterator.get_next()

    

# mean, var = tf.nn.moments(feature_batch, axes=[0])
# feature_batch = (feature_batch - mean) / tf.sqrt(var)
# mean, var = tf.nn.moments(label_batch, axes=[0])
# label_batch = (label_batch - mean) / var

(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(39)]), TensorShape([Dimension(None)]))
(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(39)]), TensorShape([Dimension(None)]))


In [3]:
# Graph definition.
dense = feature_batch
for i in range(len(net)):
    dense = tf.layers.dense(
        dense,
        net[i],
#         kernel_initializer=tf.truncated_normal_initializer(stddev=20.0),
#         kernel_initializer=tf.initializers.variance_scaling(scale=10.0),
        kernel_initializer=tf.initializers.variance_scaling(),
#         kernel_initializer=tf.initializers.orthogonal(),
#         kernel_initializer=tf.initializers.uniform_unit_scaling(),
#         kernel_initializer=tf.initializers.random_normal(stddev=10),
#         kernel_initializer=tf.initializers.random_uniform(-100, 100),
#         kernel_initializer=tf.initializers.zeros(),
#         bias_initializer=tf.initializers.zeros(),
#         activation=tf.nn.sigmoid,
        activation=tf.nn.relu,
#         activation=tf.nn.tanh,
    )
out = tf.layers.dense(
    dense,
    1,
#         kernel_initializer=tf.truncated_normal_initializer(stddev=20.0),
#         kernel_initializer=tf.initializers.variance_scaling(scale=10.0),
#         kernel_initializer=tf.initializers.orthogonal(),
#         kernel_initializer=tf.initializers.uniform_unit_scaling(),
#         kernel_initializer=tf.initializers.random_normal(stddev=10),
        kernel_initializer=tf.initializers.variance_scaling(),
#         kernel_initializer=tf.initializers.random_uniform(-100, 100),
#         kernel_initializer=tf.initializers.zeros(),
#         bias_initializer=tf.initializers.zeros(),
)


all_vars = tf.trainable_variables()
l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.0)  # TODO: scale=?
regularization_penalty = tf.contrib.layers.apply_regularization(
    l1_regularizer, all_vars)
print('out', out, 'label_batch', label_batch)
# NOTE: critical to reshape / or keep the order correct...
# out: [B, 1]
# label_batch [B]
# loss = tf.reduce_mean(tf.square(out - label_batch))
# loss = tf.reduce_mean(tf.square(out - label_batch)) 
loss = tf.reduce_mean(tf.abs(
    tf.reshape(out,[-1]) - label_batch)) + regularization_penalty

# loss = tf.reduce_mean(tf.abs(tf.reshape(out,[-1]) - label_batch))

optimizer = tf.train.AdamOptimizer(learning_rate=lr)
global_step = tf.train.get_or_create_global_step()
train_step = optimizer.minimize(
    loss, global_step=global_step)

# For visualization.
tf.summary.scalar('Train Loss', loss)
for v in all_vars:
    tf.summary.histogram(v.name, v)
all_vars_norms = [tf.norm(v) for v in all_vars]
print(len(all_vars))

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Scale of 0 disables regularizer.
out Tensor("dense_2/BiasAdd:0", shape=(?, 1), dtype=float32) label_batch Tensor("IteratorGetNext:1", shape=(?,), dtype=float32)
INFO:tensorflow:Summary name Train Loss is illegal; using Train_Loss instead.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
6


In [ ]:
# Train.
sess_args = {
#              'checkpoint_dir': os.getcwd(),
#              'save_checkpoint_secs': 60, 
             'save_summaries_secs': 1,
             'log_step_count_steps': 5000,
             'hooks': [tf.train.NanTensorHook(loss),
                       tf.train.StopAtStepHook(last_step=2000000),
#                       tf.train.LoggingTensorHook([global_step, loss], every_n_iter=500)
                      ],
            }

loss_vals = collections.deque(maxlen=40)
i = 0
with tf.train.MonitoredTrainingSession(**sess_args) as sess:
    sess.run(iterator_init)
    while not sess.should_stop():
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     running_test_loss = 0.0
#     while True:
        _, loss_val, global_step_val, true_feat_train, true_labels_train, pred_labels_train = sess.run(
            [train_step, loss, global_step,  feature_batch, label_batch, out])

        if i % 100 == 0:
#             sess.run(test_iterator_init)
#             test_loss_val, actual_labels, predicted_labels = sess.run([loss, label_batch, out])

#             loss_vals.append(test_loss_val)
#             avg_test_loss = np.mean(loss_vals)
#             print('global_step', global_step_val, ', loss', loss_val, ', avg test loss', avg_test_loss)
            print('global_step', global_step_val, ', loss', loss_val)
            if i % 1000 == 0:
#                 print(' true labels', actual_labels[:20].reshape(-1,))
#                 print(' pred labels', predicted_labels[:20].reshape(-1,))
                print(' true feats train', true_feat_train[:2])
                print(' true labels train', true_labels_train[:10].reshape(-1,))
                print(' pred labels train', pred_labels_train[:10].reshape(-1,))
                print(' all vars norms', sess.run(all_vars_norms))
        i += 1
#         if i >= 10000:
#             break

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
global_step 1 , loss 27.99059
 true feats train [[100.  10.  10.  10. 100.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   1.   0.   1.   0.   0.   0.  96.]
 [100.  10.  10.  10. 100.   0.   0.   1.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  36.]]
 true labels train [ 1.  1.  8. 10. 15.  1. 10.  8. 11. 17.]
 pred labels train [-5.848502  -4.2605777 -3.759995  -5.06429   -5.350711  -4.337441
 -3.9566202 -3.5314288 -5.9224596 -3.7954628]
 all vars norms [9.94791, 0.041827045, 9.899572, 0.044999097, 0.8957018, 0.004999998]
global_step 101 , loss 19.476458
global_step 201 , loss 14.603301
global_step 301 , loss 9.9573765
global_step 401 , loss 8.11655
global_step 501 , l